In [ ]:
!pip install optuna imbalanced-learn

# -------- core --------
import numpy as np
import pandas as pd
import pickle

# -------- optimization --------
import optuna

# -------- sklearn: preprocessing & models --------
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier

# -------- sklearn: metrics & validation --------
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report
)
from sklearn.model_selection import StratifiedKFold, cross_validate

# -------- imbalance handling --------
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# -------- XGBoost --------
from xgboost import XGBClassifier

# -------- plotting --------
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# -------- misc --------
import os
from IPython.display import Image, display, Math, Markdown

# Exploratory Data Analysis

#### Dataset retrieved from: Bertin-Mahieux, T. (2011). Year Prediction MSD [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C50K61.

In [ ]:
OUT_DIR = "images_2"

path = "YearPredictionMSD.txt"

n_features = 90
columns = ["year"] + [f"audio_feature_{i}" for i in range(1, n_features + 1)]

df = pd.read_csv(
    path,
    header=None,
    names=columns
)

df["decade"] = (df["year"] // 10) * 10

feature_cols = [c for c in df.columns if c.startswith("audio_feature_")]
df = df[["year", "decade"] + feature_cols]

display(df.columns)
display(df.head())


In [ ]:
# let's see how features compare
df.describe().T

In [ ]:
out = df.agg(["count", "mean", "median", "std", "max", "min"]).T
out["na"] = df.isna().sum()
out.head(10)


#### Fortunately, no NaN values. Clean dataset

#### Let's start with examining the year and decade distribution to see if classes are equally represented

In [ ]:
print(f"Number of distinct years in the dataset: {df.year.nunique()}")
print(f"Number of distinct decades in the dataset: {df.decade.nunique()}")

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=df["year"], name="year", opacity=0.6, nbinsx=100))
fig.add_trace(go.Histogram(x=df["decade"], name="decade", opacity=0.6, nbinsx=100))
fig.update_layout(barmode="overlay",title="Distribution of Year and Decade",xaxis_title="Year / Decade",yaxis_title="Count",legend_title="Variable")
#fig.write_image(f"{OUT_DIR}/song_count_by_year_and_decade.png")
fig.show()


In [ ]:
decade_counts = df["decade"].value_counts().sort_index()
decade_props = decade_counts / decade_counts.sum()
display(decade_counts)


In [ ]:
print(f"Decade percentages, {decade_props}")

### The decade labels in the dataset are strongly imbalanced, with approximately 60% of songs belonging to the 2000s, while earlier decades are substantially underrepresented. During modeling and evaluation, we will accordingly account for class imbalance by using evaluation metrics like macro-averaged F1 score and balanced accuracy and also by applying class weighting in the learning algorithms where supported. This ensures that our model is immune to discrimination between decades and the dominance of the majority class.

## Audio Features

### The individual features are not semantically labeled, but generally speaking, they capture global timbral and harmonic statistics sufficient for decade discrimination

### Are audio features numerically on the same scale??

In [ ]:
feature_cols = [c for c in df.columns if c.startswith("audio_feature_")]
stds = df[feature_cols].std()
display(stds.describe())


In [ ]:
# which features have the highest stds?
stds.sort_values(ascending=False).head(10)


### They are not on the same scale. So we scale them.

In [ ]:
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[feature_cols] = scaler.fit_transform(df[feature_cols])


### The heatmap below shows us that some audio features show gradual shifts across decades, suggesting systematic changes in music characteristics over time.

In [ ]:
df_agg = (df_scaled.groupby("decade")[feature_cols].mean())
fig = px.imshow(df_agg,aspect="auto",labels=dict(x="Audio feature index",y="Decade",color="Mean (z-scored)"),title="Mean Z-Scored Audio Feature Values per Decade")
#fig.write_image(f"{OUT_DIR}/Audio_Fts_per_Decade.png")
fig.show()


### The trend line below shows decade-wise trends in the scaled audio features.

### Earlier decades are characterized by substantial dispersion in feature values, with different features exhibiting markedly different average levels. As we move toward later decades, these feature trajectories progressively converge, resulting in a tighter clustering of values and reduced cross-feature variability. This suggests that songs from more recent decades share more homogeneous aggregate audio characteristics. Importantly, this convergence does not imply uniformity across features; several dimensions continue to exhibit distinct temporal trends. Rather, the figure indicates that temporal information is distributed across many features and manifests as gradual, systematic shifts rather than abrupt changes.

In [ ]:
df_feat_trend = df_scaled.groupby("decade")[feature_cols].mean().reset_index()
df_long = df_feat_trend.melt("decade", feature_cols, "feature", "value")

fig = px.line(df_long, x="decade", y="value", color="feature",
        title="Decade-wise Trends of Scaled Audio Features"
)
fig.update_layout(showlegend=False, yaxis_title="Mean scaled feature value")
#fig.write_image(f"{OUT_DIR}/Audio_Fts_TrendLine.png")
fig.show()


In [ ]:

df_feat_trend = df.groupby("decade")[feature_cols].mean().reset_index()
df_long = df_feat_trend.melt("decade", feature_cols, "feature", "value")

fig = px.line(df_long, x="decade", y="value", color="feature",
        title="Decade-wise Trends of Audio Features"
)
fig.update_layout(showlegend=False, yaxis_title="Feature value")
# fig.write_image("song_count_by_year_and_decade.png")
fig.show()


### There are 90 audio features, but are they distinct? Or are some of them redundant? If redundancy exists, we can later utilise PCA to reduce the number of features.

In [ ]:
corr = df_scaled[feature_cols].corr()
display(corr)

In [ ]:
fig = px.imshow(corr,aspect="auto",color_continuous_scale="RdBu",zmin=-1,zmax=1,title="Correlation Between Audio Features (Z-Scored)", height=900, width=900)
#fig.write_image(f"{OUT_DIR}/audio_fts_corr.png")
fig.show()

In [ ]:
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
prop_strong = (upper > 0.7).sum().sum() / upper.count().sum()
prop_strong

### Correlation analysis summary:

### Pairwise correlations between features are generally low: only about 0.2% of all feature pairs have an absolute correlation above 0.7. This indicates that most audio features capture distinct information rather than being globally redundant. However, the correlation heatmap still reveals small, localized clusters of correlated features, suggesting limited and structured redundancy within specific feature groups rather than widespread overlap across the feature set.

# Machine Learning and Classification

### Having characterized the data and its structure, we next establish a baseline decade-classification model to assess whether the audio features contain predictive temporal information.

## Baseline Model: Logistic Regression

### We begin with a simple, class-weighted multinomial logistic regression as a baseline. This model provides a conservative reference point, allowing us to assess whether the audio features contain usable decade-level information before introducing more complex models.

#### F1 vs. Accuracy

##### Accuracy measures the overall proportion of correctly classified samples. However, in this task the decade classes are imbalanced, meaning some decades appear much more frequently than others. In such settings, a model can achieve high accuracy by favoring the dominant classes while performing poorly on under-represented ones.

##### The macro-averaged F1 score addresses this issue by computing the F1 score independently for each decade and then averaging across decades. This gives equal importance to all classes, making the metric more sensitive to whether the model performs consistently across both frequent and rare decades.

##### For this reason, we report both accuracy and macro F1: accuracy reflects overall correctness, while macro F1 better reflects balanced performance across decades.

![image.png](attachment:image.png)

![image-2.png](attachment:image-2.png)




In [ ]:
from IPython.display import display, Math, Markdown

display(Markdown("## Evaluation Metrics"))

display(Markdown(
"**Accuracy** measures the overall fraction of correctly classified samples. "
"It is dominated by majority classes and can be misleading when class frequencies are highly imbalanced."
))
display(Math(r"\text{Accuracy}=\frac{1}{N}\sum_{i=1}^{N}\mathbf{1}(\hat{y}_i=y_i)"))

display(Markdown(
"**Balanced Accuracy** compensates for class imbalance by averaging recall across classes. "
"Each class contributes equally, so rare classes influence the metric as much as frequent ones."
))
display(Math(r"\text{Balanced Accuracy}=\frac{1}{K}\sum_{k=1}^{K}\frac{TP_k}{TP_k+FN_k}"))

display(Markdown(
"**F1 score** measures the balance between precision (how many predicted positives are correct) "
"and recall (how many true positives are recovered). It penalizes both false positives and false negatives."
))
display(Math(r"\text{Precision}_k=\frac{TP_k}{TP_k+FP_k}"))
display(Math(r"\text{Recall}_k=\frac{TP_k}{TP_k+FN_k}"))
display(Math(r"\text{F1}_k=\frac{2\,\text{Precision}_k\,\text{Recall}_k}{\text{Precision}_k+\text{Recall}_k}"))

display(Markdown(
"**Macro-F1** averages the F1 score independently over all classes. "
"This treats every class equally and strongly penalizes models that perform well on dominant classes "
"but poorly on rare ones."
))
display(Math(r"\text{Macro-F1}=\frac{1}{K}\sum_{k=1}^{K}\text{F1}_k"))


In [ ]:
target_ = "decade"
X = df[feature_cols]
y = df[target_]

# official UCI split -- refer to citation
X_train = X.iloc[:463715].to_numpy(dtype=np.float32)
y_train = y.iloc[:463715].to_numpy(dtype=np.float32)

X_test = X.iloc[463715:].to_numpy(dtype=np.float32)
y_test = y.iloc[463715:].to_numpy(dtype=np.float32)
labels = sorted(np.unique(y_test))

In [ ]:
len(X_train)

In [ ]:
all_accuracies = []

In [ ]:
############### Initialize model ###############
pipe_lr=Pipeline([
 ("scaler",StandardScaler()),
 ("classifier",LogisticRegression(class_weight="balanced",max_iter=2000,n_jobs=-1))
])

############### Real model ###############
pipe_lr.fit(X_train,y_train)
y_pred=pipe_lr.predict(X_test)

acc=accuracy_score(y_test,y_pred)
macro_f1=f1_score(y_test,y_pred,average="macro")
bal_acc=balanced_accuracy_score(y_test,y_pred)

############### Chance level (label shuffle) ###############
y_train_chance=np.random.permutation(y_train)

pipe_lr.fit(X_train,y_train_chance)
y_pred_chance=pipe_lr.predict(X_test)

acc_chance=accuracy_score(y_test,y_pred_chance)
macro_f1_chance=f1_score(y_test,y_pred_chance,average="macro")
bal_acc_chance=balanced_accuracy_score(y_test,y_pred_chance)

############### Print results ###############
print("Logistic Regression results")
print("Accuracy (real):",acc)
print("Balanced accuracy (real):",bal_acc)
print("Macro F1 (real):",macro_f1)
print()
print("Chance-level results (label shuffled)")
print("Accuracy (chance):",acc_chance)
print("Balanced accuracy (chance):",bal_acc_chance)
print("Macro F1 (chance):",macro_f1_chance)

all_accuracies+=[
 {"Model":"Logistic Regression","Run":"True",
  "Accuracy":acc,"Balanced Accuracy":bal_acc,"Macro F1":macro_f1},
 {"Model":"Logistic Regression","Run":"Chance",
  "Accuracy":acc_chance,"Balanced Accuracy":bal_acc_chance,"Macro F1":macro_f1_chance}
]

############### Confusion matrices ###############
cm_tr=confusion_matrix(y_test,y_pred,labels=labels,normalize="true")
cm_ch=confusion_matrix(y_test,y_pred_chance,labels=labels,normalize="true")

fig=make_subplots(1,2,subplot_titles=["Real","Chance"])
fig.add_trace(go.Heatmap(z=cm_tr,x=labels,y=labels,colorscale="Blues",
                         text=np.round(cm_tr,2),texttemplate="%{text}",showscale=True),1,1)
fig.add_trace(go.Heatmap(z=cm_ch,x=labels,y=labels,colorscale="Blues",
                         text=np.round(cm_ch,2),texttemplate="%{text}",showscale=True),1,2)
fig.update_layout(width=1500,height=800,
                  xaxis_title="Predicted decade",
                  yaxis_title="True decade")
fig.update_yaxes(autorange="reversed")
fig.show()
#fig.write_image(f"{OUT_DIR}/lr_tr_vs_chance.png")


### The baseline model performs above chance but remains very limited, indicating that decade-specific information is present but not linearly separable. This motivates the use of more expressive models in subsequent analyses.

## More advanced model: SVC (Linear)

In [ ]:
############### Initialize model ###############
pipe_svc=Pipeline([
 ("scaler",StandardScaler()),
 ("classifier",LinearSVC(class_weight="balanced",max_iter=5000))
])

############### Real model ###############
pipe_svc.fit(X_train,y_train)
y_pred=pipe_svc.predict(X_test)

acc=accuracy_score(y_test,y_pred)
macro_f1=f1_score(y_test,y_pred,average="macro")
bal_acc=balanced_accuracy_score(y_test,y_pred)

############### Chance level (label shuffle) ###############
y_train_chance=np.random.permutation(y_train)

pipe_svc.fit(X_train,y_train_chance)
y_pred_chance=pipe_svc.predict(X_test)

acc_chance=accuracy_score(y_test,y_pred_chance)
macro_f1_chance=f1_score(y_test,y_pred_chance,average="macro")
bal_acc_chance=balanced_accuracy_score(y_test,y_pred_chance)

############### Print results ###############
print("Linear SVM results")
print("Accuracy (real):",acc)
print("Balanced accuracy (real):",bal_acc)
print("Macro F1 (real):",macro_f1)
print()
print("Chance-level results (label shuffled)")
print("Accuracy (chance):",acc_chance)
print("Balanced accuracy (chance):",bal_acc_chance)
print("Macro F1 (chance):",macro_f1_chance)

all_accuracies+=[
 {"Model":"Linear SVM","Run":"True",
  "Accuracy":acc,"Balanced Accuracy":bal_acc,"Macro F1":macro_f1},
 {"Model":"Linear SVM","Run":"Chance",
  "Accuracy":acc_chance,"Balanced Accuracy":bal_acc_chance,"Macro F1":macro_f1_chance}
]

############### Confusion matrices ###############
cm_tr=confusion_matrix(y_test,y_pred,labels=labels,normalize="true")
cm_ch=confusion_matrix(y_test,y_pred_chance,labels=labels,normalize="true")

fig=make_subplots(1,2,subplot_titles=["Real","Chance"])
fig.add_trace(go.Heatmap(z=cm_tr,x=labels,y=labels,colorscale="Blues",
                         text=np.round(cm_tr,2),texttemplate="%{text}",showscale=True),1,1)
fig.add_trace(go.Heatmap(z=cm_ch,x=labels,y=labels,colorscale="Blues",
                         text=np.round(cm_ch,2),texttemplate="%{text}",showscale=True),1,2)
fig.update_layout(width=1500,height=800,
                  xaxis_title="Predicted decade",
                  yaxis_title="True decade")
fig.update_yaxes(autorange="reversed")
fig.show()
#fig.write_image(f"{OUT_DIR}/svm_tr_vs_chance.png")


## SVC with RBF Kernel

In [ ]:
# ===================== Median-capped training set =====================
counts=pd.Series(y_train).value_counts()
cap=int(counts.quantile(0.50))
print("Original class counts:")
print(counts.sort_index())
strategy={k:min(v,cap) for k,v in counts.to_dict().items()}
rus=RandomUnderSampler(sampling_strategy=strategy,random_state=0)
X_train_cap,y_train_cap=rus.fit_resample(X_train,y_train)

print("\nActual counts after resampling:")
print(pd.Series(y_train_cap).value_counts().sort_index())

# ===================== Pipeline =====================
pipe_svc=Pipeline([
 ("scaler",StandardScaler()),
 ("classifier",SVC(kernel="rbf",class_weight="balanced"))
])

# ===================== Grid  =====================
param_grid={
 "classifier__C":[1,10,100],
 "classifier__gamma":[1e-4,1e-3,1e-2]
}

grid=GridSearchCV(
 estimator=pipe_svc,
 param_grid=param_grid,
 scoring="balanced_accuracy",
 cv=2,
 n_jobs=-1,
 verbose=2
)

# ===================== Grid search =====================
grid.fit(X_train_cap,y_train_cap)

print("Best params:",grid.best_params_)
print("CV balanced accuracy:",grid.best_score_)

best_model=grid.best_estimator_

# ===================== Real model =====================
best_model.fit(X_train_cap,y_train_cap)
y_pred=best_model.predict(X_test)

acc=accuracy_score(y_test,y_pred)
macro_f1=f1_score(y_test,y_pred,average="macro")
bal_acc=balanced_accuracy_score(y_test,y_pred)

# ===================== Chance level =====================
y_train_chance=np.random.permutation(y_train_cap)

best_model.fit(X_train_cap,y_train_chance)
y_pred_chance=best_model.predict(X_test)

acc_chance=accuracy_score(y_test,y_pred_chance)
macro_f1_chance=f1_score(y_test,y_pred_chance,average="macro")
bal_acc_chance=balanced_accuracy_score(y_test,y_pred_chance)

# ===================== Log results =====================
all_accuracies+=[
 {"Model":"RBF SVM","Run":"True",
  "Accuracy":acc,"Balanced Accuracy":bal_acc,"Macro F1":macro_f1},
 {"Model":"RBF SVM","Run":"Chance",
  "Accuracy":acc_chance,"Balanced Accuracy":bal_acc_chance,"Macro F1":macro_f1_chance}
]

# ===================== Confusion matrices =====================
labels=sorted(np.unique(y_test))
cm_tr=confusion_matrix(y_test,y_pred,labels=labels,normalize="true")
cm_ch=confusion_matrix(y_test,y_pred_chance,labels=labels,normalize="true")

fig=make_subplots(1,2,subplot_titles=["Real","Chance"])
fig.add_trace(go.Heatmap(z=cm_tr,x=labels,y=labels,colorscale="Blues",
                         text=np.round(cm_tr,2),texttemplate="%{text}",showscale=True),1,1)
fig.add_trace(go.Heatmap(z=cm_ch,x=labels,y=labels,colorscale="Blues",
                         text=np.round(cm_ch,2),texttemplate="%{text}",showscale=True),1,2)
fig.update_layout(width=1500,height=800,
                  xaxis_title="Predicted decade",
                  yaxis_title="True decade")
fig.update_yaxes(autorange="reversed")
fig.show()
#fig.write_image(f"{OUT_DIR}/svm_rbf_grid_tr_vs_chance.png")


### Decent performance but not much better than our baseline (logistic) model.

## On par with SVC: Linear Discriminant Analysis

In [ ]:
############### Initialize model ###############
pipe_lda=Pipeline([
 ("scaler",StandardScaler()),
 ("classifier",LinearDiscriminantAnalysis())
])

############### Real model ###############
pipe_lda.fit(X_train,y_train)
y_pred=pipe_lda.predict(X_test)

acc=accuracy_score(y_test,y_pred)
macro_f1=f1_score(y_test,y_pred,average="macro")
bal_acc=balanced_accuracy_score(y_test,y_pred)

############### Chance level (label shuffle) ###############
y_train_chance=np.random.permutation(y_train)

pipe_lda.fit(X_train,y_train_chance)
y_pred_chance=pipe_lda.predict(X_test)

acc_chance=accuracy_score(y_test,y_pred_chance)
macro_f1_chance=f1_score(y_test,y_pred_chance,average="macro")
bal_acc_chance=balanced_accuracy_score(y_test,y_pred_chance)

############### Print results ###############
print("LDA results")
print("Accuracy (real):",acc)
print("Balanced accuracy (real):",bal_acc)
print("Macro F1 (real):",macro_f1)
print()
print("Chance-level results (label shuffled)")
print("Accuracy (chance):",acc_chance)
print("Balanced accuracy (chance):",bal_acc_chance)
print("Macro F1 (chance):",macro_f1_chance)

all_accuracies+=[
 {"Model":"LDA","Run":"True",
  "Accuracy":acc,"Balanced Accuracy":bal_acc,"Macro F1":macro_f1},
 {"Model":"LDA","Run":"Chance",
  "Accuracy":acc_chance,"Balanced Accuracy":bal_acc_chance,"Macro F1":macro_f1_chance}
]

############### Confusion matrices ###############
labels=np.unique(y_test)
cm_tr=confusion_matrix(y_test,y_pred,labels=labels,normalize="true")
cm_ch=confusion_matrix(y_test,y_pred_chance,labels=labels,normalize="true")

fig=make_subplots(1,2,subplot_titles=["Real","Chance"])
fig.add_trace(go.Heatmap(z=cm_tr,x=labels,y=labels,colorscale="Blues",
                         text=np.round(cm_tr,2),texttemplate="%{text}",showscale=True),1,1)
fig.add_trace(go.Heatmap(z=cm_ch,x=labels,y=labels,colorscale="Blues",
                         text=np.round(cm_ch,2),texttemplate="%{text}",showscale=True),1,2)
fig.update_layout(width=1500,height=800,
                  xaxis_title="Predicted decade",
                  yaxis_title="True decade")
fig.update_yaxes(autorange="reversed")
fig.show()
#fig.write_image(f"{OUT_DIR}/lda_tr_vs_chance.png")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

############### Hyperparameter Optimization with Subset ###############
# Use subset for faster optimization
SUBSET_SIZE = 50000
np.random.seed(0)
subset_indices = np.random.choice(len(X_train), size=SUBSET_SIZE, replace=False)
X_subset = X_train[subset_indices]
y_subset = y_train[subset_indices]

def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 10, 40),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
        'n_jobs': -1,
        'random_state': 0
    }

    rf = RandomForestClassifier(**params)

    # 3-fold CV on macro F1
    scores = cross_val_score(rf, X_subset, y_subset, cv=3, scoring='f1_macro', n_jobs=1)

    return scores.mean()

# Create and run study
print("Starting Random Forest hyperparameter optimization...")
print(f"Using subset of {SUBSET_SIZE} samples for faster tuning")

optuna_already_run=True

if not optuna_already_run:
    study_rf=optuna.create_study(direction="maximize", study_name='rf_msd')
    study_rf.optimize(objective_rf,n_trials=50, show_progress_bar=True)
    with open("study_rf.pkl","wb") as f:
        pickle.dump(study_rf,f)
else:
    with open("study_rf.pkl","rb") as f:
        study_rf=pickle.load(f)



print("\nBest parameters:", study_rf.best_params)
print("Best macro F1 (on subset CV):", study_rf.best_value)



############### Initialize model ###############
best_params_rf = study_rf.best_params
best_params_rf.update({
    "n_jobs": -1,
    "random_state": 0
})

rf = RandomForestClassifier(**best_params_rf)

############### Real model (trained on FULL training set) ###############
print("\nTraining on full training set...")
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average="macro")
bal_acc = balanced_accuracy_score(y_test, y_pred)

############### Chance level ###############
print("Training chance model...")
ytr_chance = np.random.permutation(y_train)
rf.fit(X_train, ytr_chance)
y_pred_chance = rf.predict(X_test)

acc_chance = accuracy_score(y_test, y_pred_chance)
macro_f1_chance = f1_score(y_test, y_pred_chance, average="macro")
bal_acc_chance = balanced_accuracy_score(y_test, y_pred_chance)

############### Print results ###############
print("\n" + "="*50)
print("RANDOM FOREST RESULTS")
print("="*50)
print("Accuracy (real):", acc)
print("Balanced accuracy (real):", bal_acc)
print("Macro F1 (real):", macro_f1)
print()
print("Accuracy (chance):", acc_chance)
print("Balanced accuracy (chance):", bal_acc_chance)
print("Macro F1 (chance):", macro_f1_chance)

all_accuracies += [
    {"Model": "Random Forest", "Run": "True", "Accuracy": acc, "Balanced Accuracy": bal_acc, "Macro F1": macro_f1},
    {"Model": "Random Forest", "Run": "Chance", "Accuracy": acc_chance, "Balanced Accuracy": bal_acc_chance, "Macro F1": macro_f1_chance}
]

############### Confusion matrices ###############
labels = sorted(np.unique(y_test))

cm_tr = confusion_matrix(y_test, y_pred, labels=labels, normalize="true")
cm_ch = confusion_matrix(y_test, y_pred_chance, labels=labels, normalize="true")

fig = make_subplots(1, 2, subplot_titles=["Real", "Chance"])
fig.add_trace(go.Heatmap(z=cm_tr, x=labels, y=labels, colorscale="Blues",
                         text=np.round(cm_tr, 2), texttemplate="%{text}", showscale=True), 1, 1)
fig.add_trace(go.Heatmap(z=cm_ch, x=labels, y=labels, colorscale="Blues",
                         text=np.round(cm_ch, 2), texttemplate="%{text}", showscale=True), 1, 2)
fig.update_layout(width=1500, height=800,
                  xaxis_title="Predicted decade", yaxis_title="True decade",
                  xaxis2_title="Predicted decade")
fig.update_yaxes(autorange="reversed")
fig.show()

fig.write_image(f"{OUT_DIR}/rf_tr_vs_chance.png")

############### Feature Importance ###############
# Train final model for feature importance
rf.fit(X_train, y_train)
feature_importance = rf.feature_importances_

# Sort features by importance
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(importance_df.head(10))

# Optional: Plot feature importance
fig_imp = go.Figure(go.Bar(
    x=importance_df['importance'].head(20),
    y=importance_df['feature'].head(20),
    orientation='h'
))
fig_imp.update_layout(
    title="Top 20 Feature Importances (Random Forest)",
    xaxis_title="Importance",
    yaxis_title="Feature",
    height=600,
    yaxis={'categoryorder': 'total ascending'}
)
fig_imp.show()
#fig_imp.write_image(f"{OUT_DIR}/rf_feature_importance.png")

### Given the poor accuracy results here, we will opt for a more advanced model: XGBoost.

## More advanced model: XGBoost

### We next evaluate a gradient-boosted decision tree model (XGBoost), which is well suited to capturing nonlinear interactions in high-dimensional tabular audio features.

### We initially ran a grid search to find the optimal parameters for classification. However, given how computationally expensive the search was, we stopped the kernel after 56 hours and opted for a more efficient library: Optuna. Grid search evaluates all combinations of parameters to evaluate which drives best classification results; however, Optuna consistenly evaluates combinations that are more likely to improve classification performance, thereby cutting down the amount of time needed to converge on an ideal combination.

In [ ]:
# ---------------- CONFIG ----------------
USE_GPU=False
N_TRIALS=100
CV_FOLDS=5
N_CORES=8
# --------------------------------------

# -------- XGBOOST DATA VIEW -----------
from sklearn.preprocessing import LabelEncoder

le_xgb=LabelEncoder()
ytr_xgb=le_xgb.fit_transform(y_train)
yte_xgb=le_xgb.transform(y_test)

Xtr_xgb=X_train
Xte_xgb=X_test
# --------------------------------------

def objective(trial):
    params={
        "n_estimators":trial.suggest_int("n_estimators",300,900),
        "max_depth":trial.suggest_int("max_depth",3,7),
        "learning_rate":trial.suggest_float("learning_rate",0.02,0.1,log=True),
        "subsample":trial.suggest_float("subsample",0.6,0.9),
        "colsample_bytree":trial.suggest_float("colsample_bytree",0.4,0.8),
        "min_child_weight":trial.suggest_int("min_child_weight",1,10),
        "reg_lambda":trial.suggest_float("reg_lambda",1.0,10.0),
        "reg_alpha":trial.suggest_float("reg_alpha",0.0,1.0),
        "objective":"multi:softprob",
        "eval_metric":"mlogloss",
        "n_jobs":1,
        "random_state":0,
        "tree_method":"gpu_hist" if USE_GPU else "hist",
        "device":"cuda" if USE_GPU else None
    }

    model=XGBClassifier(**params)
    cv=StratifiedKFold(n_splits=CV_FOLDS,shuffle=True,random_state=0)

    scores=[]
    for tr,va in cv.split(Xtr_xgb,ytr_xgb):
        model.fit(Xtr_xgb[tr],ytr_xgb[tr])
        pred=model.predict(Xtr_xgb[va])
        scores.append(f1_score(ytr_xgb[va],pred,average="macro"))

    return np.mean(scores)

optuna_already_run=True

if not optuna_already_run:
    study=optuna.create_study(direction="maximize")
    study.optimize(objective,n_trials=N_TRIALS,n_jobs=N_CORES)
    with open("study.pkl","wb") as f:
        pickle.dump(study,f)
else:
    with open("study.pkl","rb") as f:
        study=pickle.load(f)


### Using the ideal parameters from the Optuna search, we are able to classify song decade based on audio features with 65% accuracy. Chance level is 10%.

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
############### Initialize model ###############
best_params = study.best_params
best_params.update({
    "objective": "multi:softprob",
    "eval_metric": "mlogloss",
    "n_jobs": -1,
    "random_state": 0,
    "tree_method": "gpu_hist" if USE_GPU else "hist",
    "device": "cuda" if USE_GPU else None
})

xgb = XGBClassifier(**best_params)




############### Real model ###############
xgb.fit(Xtr_xgb, ytr_xgb)
y_pred = xgb.predict(Xte_xgb)

acc = accuracy_score(yte_xgb,y_pred)
macro_f1 = f1_score(yte_xgb,y_pred,average="macro")
bal_acc = balanced_accuracy_score(yte_xgb,y_pred)
############### Chance level ###############
ytr_chance = np.random.permutation(ytr_xgb)
xgb.fit(Xtr_xgb, ytr_chance)
y_pred_chance = xgb.predict(Xte_xgb)

acc_chance = accuracy_score(yte_xgb,y_pred_chance)
macro_f1_chance = f1_score(yte_xgb,y_pred_chance,average="macro")
bal_acc_chance = balanced_accuracy_score(yte_xgb,y_pred_chance)
############### Print results ###############
print("Accuracy (real):",acc)
print("Balanced accuracy (real):",bal_acc)
print("Macro F1 (real):",macro_f1)
print()
print("Accuracy (chance):",acc_chance)
print("Balanced accuracy (chance):",bal_acc_chance)
print("Macro F1 (chance):",macro_f1_chance)


all_accuracies += [
{"Model":"XGBoost","Run":"True","Accuracy":acc,"Balanced Accuracy":bal_acc,"Macro F1":macro_f1},
{"Model":"XGBoost","Run":"Chance","Accuracy":acc_chance,"Balanced Accuracy":bal_acc_chance,"Macro F1":macro_f1_chance}

]

############### Confusion matrices ###############
labels = np.arange(len(le_xgb.classes_))

cm_tr = confusion_matrix(yte_xgb, y_pred, labels=labels, normalize="true")
cm_ch = confusion_matrix(yte_xgb, y_pred_chance, labels=labels, normalize="true")

fig = make_subplots(1, 2, subplot_titles=["Real", "Chance"])
fig.add_trace(go.Heatmap(z=cm_tr,x=labels,y=labels,colorscale="Blues",text=np.round(cm_tr,2),texttemplate="%{text}",showscale=True),1,1)
fig.add_trace(go.Heatmap(z=cm_ch,x=labels,y=labels,colorscale="Blues",text=np.round(cm_ch,2),texttemplate="%{text}",showscale=True),1,2)
fig.update_layout(width=1500, height=800,xaxis_title="Predicted class (ordinal index)",yaxis_title="True class (ordinal index)")
fig.update_yaxes(autorange="reversed")
fig.show()

#fig.write_image(f"{OUT_DIR}/xgb_tr_vs_chance.png")


In [ ]:
############### Initialize model ###############
counts=pd.Series(ytr_xgb).value_counts()
cap=int(counts.quantile(0.5))
strategy={k:min(v,cap) for k,v in counts.to_dict().items()}
rus=RandomUnderSampler(sampling_strategy=strategy,random_state=0)
Xtr_cap,ytr_cap=rus.fit_resample(Xtr_xgb,ytr_xgb)

best_params=study.best_params
best_params.update({
 "objective":"multi:softprob","eval_metric":"mlogloss",
 "n_jobs":-1,"random_state":0,
 "tree_method":"gpu_hist" if USE_GPU else "hist",
 "device":"cuda" if USE_GPU else None
})

xgb=XGBClassifier(**best_params)

############### Real model ###############

xgb.fit(Xtr_cap,ytr_cap)
y_pred=xgb.predict(Xte_xgb)

acc=accuracy_score(yte_xgb,y_pred)
macro_f1=f1_score(yte_xgb,y_pred,average="macro")
bal_acc=balanced_accuracy_score(yte_xgb,y_pred)

############### Chance level ###############

ytr_cap_chance=np.random.permutation(ytr_cap)

xgb.fit(Xtr_cap,ytr_cap_chance)
y_pred_chance=xgb.predict(Xte_xgb)
acc_chance=accuracy_score(yte_xgb,y_pred_chance)
macro_f1_chance=f1_score(yte_xgb,y_pred_chance,average="macro")
bal_acc_chance=balanced_accuracy_score(yte_xgb,y_pred_chance)
############### Print results ###############
print("Accuracy (real):",acc)
print("Balanced accuracy (real):",bal_acc)
print("Macro F1 (real):",macro_f1)
print()
print("Accuracy (chance):",acc_chance)
print("Balanced accuracy (chance):",bal_acc_chance)
print("Macro F1 (chance):",macro_f1_chance)


all_accuracies+=[
{"Model":"XGBoost (capped)","Run":"True","Accuracy":acc,"Balanced Accuracy":bal_acc,"Macro F1":macro_f1},
{"Model":"XGBoost (capped)","Run":"Chance","Accuracy":acc_chance,"Balanced Accuracy":bal_acc_chance,"Macro F1":macro_f1_chance}
]

############### Confusion matrices ###############

labels=np.arange(len(le_xgb.classes_))
cm_tr=confusion_matrix(yte_xgb,y_pred,labels=labels,normalize="true")
cm_ch=confusion_matrix(yte_xgb,y_pred_chance,labels=labels,normalize="true")

fig=make_subplots(1,2,subplot_titles=["Real","Chance"])
fig.add_trace(go.Heatmap(z=cm_tr,x=labels,y=labels,colorscale="Blues",text=np.round(cm_tr,2),texttemplate="%{text}",showscale=True),1,1)
fig.add_trace(go.Heatmap(z=cm_ch,x=labels,y=labels,colorscale="Blues",text=np.round(cm_ch,2),texttemplate="%{text}",showscale=True),1,2)
fig.update_layout(width=1500,height=800,
                  xaxis_title="Predicted class (ordinal index)",
                  yaxis_title="True class (ordinal index)")
fig.update_yaxes(autorange="reversed")
fig.show()

#fig.write_image(f"{OUT_DIR}/xgb_capped_tr_vs_chance.png")


In [ ]:
# ordinal encoding from reference labels
decades_sorted=np.sort(np.unique(y_train))
dec2i={d:i for i,d in enumerate(decades_sorted)}

ytr_ord=np.array([dec2i[d] for d in y_train])
yte_ord=np.array([dec2i[d] for d in y_test])

ord_model=Pipeline([
 ("scaler",StandardScaler()),
 ("regressor",Ridge(alpha=1.0))
])

ord_model.fit(X_train,ytr_ord)
y_pred_cont=ord_model.predict(X_test)
y_pred=np.clip(np.rint(y_pred_cont),0,len(decades_sorted)-1).astype(int)

acc=accuracy_score(yte_ord,y_pred)
macro_f1=f1_score(yte_ord,y_pred,average="macro")
bal_acc=balanced_accuracy_score(yte_ord,y_pred)

ytr_chance=np.random.permutation(ytr_ord)
ord_model.fit(X_train,ytr_chance)
y_pred_chance=np.clip(
 np.rint(ord_model.predict(X_test)),
 0,len(decades_sorted)-1
).astype(int)

acc_chance=accuracy_score(yte_ord,y_pred_chance)
macro_f1_chance=f1_score(yte_ord,y_pred_chance,average="macro")
bal_acc_chance=balanced_accuracy_score(yte_ord,y_pred_chance)

print("Accuracy (real):",acc)
print("Balanced accuracy (real):",bal_acc)
print("Macro F1 (real):",macro_f1)
print()
print("Accuracy (chance):",acc_chance)
print("Balanced accuracy (chance):",bal_acc_chance)
print("Macro F1 (chance):",macro_f1_chance)



all_accuracies+=[
{"Model":"Ordinal Ridge","Run":"True","Accuracy":acc,"Balanced Accuracy":bal_acc,"Macro F1":macro_f1},
{"Model":"Ordinal Ridge","Run":"Chance","Accuracy":acc_chance,"Balanced Accuracy":bal_acc_chance,"Macro F1":macro_f1_chance}
]

labels=np.arange(len(decades_sorted))
cm_tr=confusion_matrix(yte_ord,y_pred,labels=labels,normalize="true")
cm_ch=confusion_matrix(yte_ord,y_pred_chance,labels=labels,normalize="true")

fig=make_subplots(1,2,subplot_titles=["Real","Chance"])
fig.add_trace(go.Heatmap(z=cm_tr,x=labels,y=labels,colorscale="Blues",text=np.round(cm_tr,2),texttemplate="%{text}",showscale=True),1,1)
fig.add_trace(go.Heatmap(z=cm_ch,x=labels,y=labels,colorscale="Blues",text=np.round(cm_ch,2),texttemplate="%{text}",showscale=True),1,2)
fig.update_yaxes(autorange="reversed")
fig.show()

#fig.write_image(f"{OUT_DIR}/ordinal_ridge_tr_vs_chance.png")


In [ ]:
df=pd.DataFrame(all_accuracies)
df.to_csv(f"{OUT_DIR}/all_model_results.csv",index=False)

# Conclusion

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

METRICS=["Accuracy","Balanced Accuracy","Macro F1"]

# ---------- auto models ----------
models=sorted(df["Model"].unique())

# ---------- auto colors ----------
palette=px.colors.qualitative.Plotly
model_colors={m:palette[i%len(palette)] for i,m in enumerate(models)}

def hex_to_rgba(hex_color,alpha):
    h=hex_color.lstrip("#")
    r,g,b=(int(h[i:i+2],16) for i in (0,2,4))
    return f"rgba({r},{g},{b},{alpha})"

for METRIC_COL in METRICS:

    title=f"{METRIC_COL} by Model"
    out_file=METRIC_COL.lower().replace(" ","_")+"_by_model.png"

    y_true=[df[(df.Model==m)&(df.Run=="True")][METRIC_COL].values[0] for m in models]
    y_ch=[df[(df.Model==m)&(df.Run=="Chance")][METRIC_COL].values[0] for m in models]

    fig=go.Figure()

    fig.add_bar(
        x=models,y=y_true,offsetgroup="true",
        marker_color=[model_colors[m] for m in models],
        name="True labels"
    )

    fig.add_bar(
        x=models,y=y_ch,offsetgroup="shuffled",
        marker_color=[hex_to_rgba(model_colors[m],0.35) for m in models],
        name="Shuffled labels"
    )

    fig.update_layout(
        barmode="group",
        title=title,
        yaxis_title=METRIC_COL,
        xaxis_title="Model",
        legend_title="",
        width=1500,
        height=700
    )

    fig.show()
    #fig.write_image(f"{OUT_DIR}/{out_file}")


In [ ]:
import plotly.graph_objects as go
import plotly.express as px

METRICS = ["Accuracy", "Balanced Accuracy", "Macro F1"]

# ---------- auto models ----------
models = sorted(df["Model"].unique())

# ---------- auto colors ----------
palette = px.colors.qualitative.Plotly
model_colors = {m: palette[i % len(palette)] for i, m in enumerate(models)}

def hex_to_rgba(hex_color, alpha):
    h = hex_color.lstrip("#")
    r, g, b = (int(h[i:i+2], 16) for i in (0, 2, 4))
    return f"rgba({r},{g},{b},{alpha})"

for METRIC_COL in METRICS:

    title = f"{METRIC_COL} by Model"
    out_file = METRIC_COL.lower().replace(" ", "_") + "_by_model.svg"

    y_true = [df[(df.Model == m) & (df.Run == "True")][METRIC_COL].values[0] for m in models]
    y_ch = [df[(df.Model == m) & (df.Run == "Chance")][METRIC_COL].values[0] for m in models]

    fig = go.Figure()

    fig.add_bar(
        x=models, y=y_true, offsetgroup="true",
        marker_color=[model_colors[m] for m in models],
        name="True labels"
    )

    fig.add_bar(
        x=models, y=y_ch, offsetgroup="shuffled",
        marker_color=[hex_to_rgba(model_colors[m], 0.35) for m in models],
        name="Shuffled labels"
    )

    fig.update_layout(
        barmode="group",
        title={
            'text': title,
            'font': {'size': 32}  # Larger title
        },
        yaxis_title=METRIC_COL,
        xaxis_title="Model",
        legend_title="",
        width=1500,
        height=700,
        # Increased font sizes for all text elements
        font=dict(size=18),  # Base font size for all text
        xaxis=dict(
            tickfont=dict(size=16),
            title=dict(font=dict(size=20))  # Correct syntax for axis title font
        ),
        yaxis=dict(
            tickfont=dict(size=16),
            title=dict(font=dict(size=20))  # Correct syntax for axis title font
        ),
        legend=dict(
            font=dict(size=16)
        )
    )

    fig.show()

    # Save as SVG for perfect scalability in Word
    # fig.write_image(
    #     f"{OUT_DIR}/{out_file}",
    #     format="svg",
    #     width=1500,
    #     height=700
    # )

# Appendix

#### In case images do not render, I have displayed them below